# Работа со строковыми значениями

Материалы:
* Макрушин С.В. Лекция 8: Работа со строковыми значениям
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
* https://tproger.ru/translations/regular-expression-python/
* https://realpython.com/nltk-nlp-python/

In [42]:
import re
import pandas as pd

In [8]:
import nltk

In [10]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lisa\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [41]:
obj = {  
         'home_page': 'https://github.com/pypa/sampleproject',
         'keywords': 'sample setuptools development',
         'license': 'MIT'
      }
for k, v in obj.items():
      print(f'{k:10} = "{v}"')

home_page  = "https://github.com/pypa/sampleproject"
keywords   = "sample setuptools development"
license    = "MIT"


2. Дана строка 'aaa--bbb==ccc__ddd'. Написать регулярное выражение для разбивки строки на список ['aaa','bbb','ccc','ddd'].

In [22]:
pattern = re.compile(r'[a-z]{3}')
text = 'aaa--bbb==ccc__ddd'
pattern.findall(text)

['aaa', 'bbb', 'ccc', 'ddd']

3. Проверить корректность введенного E-mail

In [38]:
email = 'NVBlokhin@fa.ru'
pattern = re.compile(r'\w+@[a-z]+\.[a-z]{2,3}$')
pattern.match(email)

<re.Match object; span=(0, 15), match='NVBlokhin@fa.ru'>

4. Разбейте текст формулировки задачи 1 на слова.

In [18]:
text = """Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки."""
nltk.word_tokenize(text)[:5]

['Вывести', 'на', 'экран', 'данные', 'из']

In [19]:
nltk.sent_tokenize(text)

['Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках.',
 'Строковые литералы обернуть в кавычки.']

## Лабораторная работа 8

### Форматирование строк

1.1 Загрузите данные из файла `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в таблицу `recipes` При помощи форматирования строк выведите информацию об id рецепта и кол-ве игредиентов 5 случайных рецептов в виде таблицы следующего вида:

    
    |    id     | n_in  |
    |-------------------|
    |  400894   |  13   |
    |   68588   |   8   |
    |  362081   |   6   |
    |   53408   |  12   |
    |  221203   |   4   |

In [45]:
recipes = pd.read_csv('recipes_sample_with_tags_ingredients.csv', parse_dates=['submitted'], index_col=0)
rec = recipes.sample(5)
rec

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients,tags,n_tags,indredients
1662,bacon fried rice,52190,30,68453,2003-01-24,6,awesome recipe! i can't eat rice any other way...,6,30-minutes-or-less;time-to-make;main-ingredien...,16,garlic powder*green onions*soy sauce*egg*bacon...
7340,coffee chocolate smoothie,129180,3,227010,2005-07-09,1,a lovely chocolaty but yet coffee like smoothie.,6,15-minutes-or-less;time-to-make;course;main-in...,23,cream*sugar*chocolate syrup*ice*coffee*chocola...
26535,swedish ikea meatballs,525547,40,1802649943,2016-01-12,14,"these homemade meatballs, smothered in a cream...",15,60-minutes-or-less;time-to-make;course;main-in...,17,onion*butter*sour cream*all-purpose flour*brea...
24072,shrimp dip ii,73056,10,87877,2003-10-10,4,i got this recipe from my boss's wife close to...,11,15-minutes-or-less;time-to-make;course;main-in...,21,worcestershire sauce*tabasco sauce*milk*fresh ...
12523,greek apple salad with pine nuts and feta,76368,10,80353,2003-11-13,6,a very refreshing salad to be enjoyed any time...,11,15-minutes-or-less;time-to-make;course;main-in...,23,sugar*olive oil*dijon mustard*red onion*pine n...


In [119]:
print(f"| {'id':^10} | {'n_in':^8} |")
print(f"|{'':-^23}|")
for i, line in rec.iterrows():
      print(f"| {line['id']:^10} | {line['n_ingredients']:^8} |")

|     id     |   n_in   |
|-----------------------|
|   52190    |    6     |
|   129180   |    6     |
|   525547   |    15    |
|   73056    |    11    |
|   76368    |    11    |


1.2 Напишите функцию `show_info`, которая для рецепта по его `id` создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название"

1. Шаг 1.
2. Шаг 2.
----------
#тэг1 #тэг2
```

    
Данные для создания строки получите из файлов `recipes_sample_with_tags_ingredients.csv`, `steps_sample.xml` (__ЛР4__) и `tags_sample.csv` (__ЛР5__). 
Выведите созданную строку на экран.

## Работа с регулярными выражениями

В задачах данного блока подразумевается, что вы не будете использовать никаких строковые методы (`split`, `startswith` и т.д.). Все задачи необходимо решить при помощи регулярных выражений.

2.1 Посчитайте кол-во отзывов, в которых встречаются числа.

2.2 Найдите все смайлики в отзывах. Смайлик состоит из трех частей: глаза (символ `=` или `:`), нос (символ `-`), губы (символ `)` или `(`). Смайлик может иметь вид "глаза-нос-губы" или "губы-нос-глаза". Нос может отсутствовать.

2.3 Проверьте, что все даты в датасете имеют вид "YYYY-MM-DD". Продемонстрируйте работу вашего решения, приведя пример из датасета и контрпример не из датасета.

2.4 Используя строку-результат задачи 1.2, найдите первое слово каждого шага в рецепте

2.5 Используя регулярные выражения, удалите из описаний все символы, кроме английских букв, цифр и пробелов. Сохраните предобработанные описания в файл `preprocessed_descriptions.csv`, содержащий 2 столбца: `name` и `preprocessed_descriptions`.

### Сегментация текста

3.1 Разбейте отзывы из файла `recipes_sample.csv` (__ЛР2__) на предложения, а предложения - на слова (используйте `sent_tokenize` и `word_tokenize` из `nltk`). Каждый отзыв представьте в виде списка списков: внешний список - предложения, вложенные списки - слова в предложении. Исключите знаки препинания из списков слов.

`'Предложение номер один. Предложение номер два.' => [['Предложение', 'номер', 'один'], ['Предложение', 'номер', 'два']]`

3.2 Посчитайте кол-во уникальных слов среди всех отзывов (без учета регистра и знаков препинания).

3.3 Найдите 5 самых длинных (по количеству слов; без учета знаков препинания) отзывов в датасете и выведите их в порядке убывания длины.

3.4 Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на любом предложении из отзывов.


#### [версия 2]
* Уточнены формулировки задач 3.1-3.3